### RNN

In [ ]:
import numpy as np
import pandas as pd

import matplotlib # Para ver la versión
import matplotlib.pyplot as plt

import tensorflow as tf # Para ver la versión
import keras # Para ver la versión

from keras.models import Sequential
from keras.layers import Input, Dense, SimpleRNN, Dropout, LSTM

import sklearn # Para ver la versión

from sklearn.preprocessing import MinMaxScaler

plt.rcParams["figure.figsize"] = (20, 8)

In [ ]:
# Versiones

print(f"numpy=={np.__version__}")
print(f"pandas=={pd.__version__}")
print(f"matplotlib=={matplotlib.__version__}")
print(f"tensorflow=={tf.__version__}")
print(f"keras=={keras.__version__}")
print(f"scikit-learn=={sklearn.__version__}")

# Unidades de procesamiento
print("\nUnidades de procesamiento:")
for device in tf.config.list_physical_devices():
    print(f"\t- {device.name}")

In [ ]:
df = pd.read_csv("../Data/power_usage_2016_to_2020.csv")

df

In [ ]:
df.info()

In [ ]:
# Preprocesamiento

datos = df.drop(["StartDate", "day_of_week", "notes"], axis = 1)

scaler = MinMaxScaler()
datos = scaler.fit_transform(datos)


datos.shape

In [ ]:
plt.plot(datos)
plt.show()

In [ ]:
df["Year"] = df["StartDate"].map(lambda x : x[:4]) # obtenemos la columna año
df.head()

In [ ]:
df = df[df["Year"] == "2019"] #Filtramos por el año 2019
df.reset_index(drop = True, inplace = True)
df

In [ ]:
# Preprocesamiento

datos = df.drop(["StartDate", "day_of_week", "notes", "Year"], axis = 1)

scaler = MinMaxScaler()
datos = scaler.fit_transform(datos)

datos.shape

In [ ]:
plt.plot(datos)
plt.show()

In [ ]:
df["day_month"] = df["StartDate"].map(lambda x : x[5: 10]).values
df.head(30)

In [ ]:
datos = df.groupby("day_month")["Value (kWh)"].sum().values

datos

In [ ]:
scaler = MinMaxScaler()

datos = scaler.fit_transform(datos.reshape(-1, 1))

datos.shape

In [ ]:
plt.plot(datos)
plt.show()

In [ ]:
T = 20 # Segmentos
X = list()
y = list()

for t in range(len(datos) - T):
    
    # Toma valores de X de t en t con stride de 1
    x = datos[t : t + T]
    X.append(x)
    
    # Toma los valores de t en t
    y_ = datos[t + T]
    y.append(y_)

# Transformamos a np.array y ajustamos las dimensiones
# Para entrar en el modelo debe de tener 3 dimensiones
X = np.array(X).reshape(-1, T, 1)
y = np.array(y)
N = len(X)

print(f"X: {X.shape}\ty: {y.shape}")

### RNN

In [ ]:
model = Sequential()

model.add(Input(shape = (T, 1)))

model.add(LSTM(units = 200, activation = "relu"))

model.add(Dense(units = 64, activation = "relu")) 
model.add(Dense(units = 32, activation = "relu")) 
model.add(Dense(units = 16, activation = "relu")) 

model.add(Dense(units = 1))

model.compile(optimizer = "adam", loss = "mse")

model.summary()

In [ ]:
history = model.fit(x = X[:300],
                    y = y[:300],
                    validation_data = (X[300:], y[300:]),
                    epochs = 1_000)

In [ ]:
# loss
plt.plot(history.history["loss"], label = "loss")
plt.plot(history.history["val_loss"], label = "val_loss")
plt.legend()
# plt.ylim(0, 1)
plt.show()

### Predicciones

In [ ]:
# "1 - Step Predictions"

# Toma un valor y predice el siguiente

validation_target = y[300:]
validation_predictions = list()

i = 300

while len(validation_predictions) < len(validation_target):
    
    # Predice el siguiente valor de X[i]
    p = model.predict(X[i].reshape(1, -1, 1))[0, 0]
    i += 1
    
    validation_predictions.append(p)


In [ ]:
plt.plot(validation_target, label = "forecast target")
plt.plot(validation_predictions, label = "forecast prediction")
plt.legend()
plt.show()

In [ ]:
# "Multiple - Step Predictions"

# Toma el último valor de una serie y predice el siguiente
# Usa esa predicción para seguir haciendo predicciones.

validation_target = y[300:]
validation_predictions = list()

last_x = X[300]

while len(validation_predictions) < len(validation_target):
    
    # En la primera iteración predice el siguiente valor de usando X
    # En las siguientes iteraciones usa el valor predicho anterior para predecir el siguiente
    p = model.predict(last_x.reshape(1, -1, 1))[0, 0]
    
    validation_predictions.append(p)
    print(f"Valor: {last_x[-1][0]}\tPredicción: {p}")
    # Desplaza los elementos en last_x hacia atras, dejando el primer elemento al final
    last_x = np.roll(last_x, -1)
    
    # Cambia el último elemento a la predicción
    last_x[-1] = p

In [ ]:
plt.plot(validation_target, label = "forecast target")
plt.plot(validation_predictions, label = "forecast prediction")
plt.legend()
plt.show()

In [ ]:
################################################################################################################################